In [ ]:
from collections import defaultdict
import json, math, glob
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.express as px
from tqdm import tqdm

def get_cruxeval(type):
    records = []
    for fname in glob.glob(f"evaluation_results/*_temp0.2_{type}.json"):
        name = fname.split('/')[1]
        model, temp, typejsonl = name.split('_')
        print(model, temp, type)

        with open(fname) as f:
            res = json.load(f)['raw_scored_generations']
            for exid in res:
                gotid = np.mean(res[exid])
                actualid = exid.replace('sample_', '')
                records.append({
                    'benchmark_id': f'CRUXEval-{type}',
                    'model': model,
                    'example_id': f"CRUXEval-{type}/{actualid}",
                    'pass1': gotid,
                    'hyperparams': 'temp0.2'
                })
    df = pd.DataFrame(records)
    display(df.describe())
    return df
        
with open('data/cruxeval_input.jsonl', 'w') as f:
    dfi = get_cruxeval('input')
    f.write(dfi.to_json(orient='records', lines=True))

with open('data/cruxeval_output.jsonl', 'w') as f:
    dfo = get_cruxeval('output')
    f.write(dfo.to_json(orient='records', lines=True))

pass1 = pd.concat([dfi, dfo])
display(pass1)
